In [1]:
using Erdos
using Distributions
using Gadfly
using Clustering
using MatrixNetworks

g = readgraph(:condmat);
a = adjacency_list(g::AGraph);

N = size(a)[1];
A0  = Array{Int64,2}(N,N);
for i=1:N
    for j=1:N
            A0[i,j]=0;
    end
end
adj = [];

for i=1:N
    l = a[i];
    for j=1:length(l)
        A0[i,l[j]]=1;
        adj = [adj;[i l[j]]];
    end
end
#A0 = (A0+A0');
A0 = sparse(A0);
A,p = largest_component(A0);
A0=0;p=0;
gc();


In [2]:
d = sum(A,2);
s = sum(d);

r = 100;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 

print(l0-shift)

[23.4192, 21.4179, 19.7087, 19.3216, 18.8043, 18.2802, 17.6249, 17.145, 16.7283, 16.4771, 16.2673, 16.2364, 16.1282, 15.7916, 15.4302, 15.2722, 15.2381, 15.1715, 14.8601, 14.8144, 14.455, 14.3304, 14.2502, 14.2218, 14.0597, 13.9875, 13.7632, 13.7579, 13.7056, 13.5241, 13.2759, 12.9008, 12.8856, 12.5924, 12.376, 12.3588, 12.3308, 12.2481, 12.1951, 11.9976, 12.0518, 11.8493, 11.8524, 11.7992, 11.6195, 11.5613, 11.5756, 11.4933, 11.4419, 11.366, 11.3384, 11.3318, 11.3226, 11.1746, 11.123, 11.1423, 10.9476, 11.0432, 10.9495, 10.9156, 10.9051, 10.8786, 10.8467, 10.7213, 10.7699, 10.6716, 10.6144, 10.5923, 10.5501, 10.5561, 10.5109, 10.4998, 10.4424, 10.4692, 10.4036, 10.4339, 10.4108, 10.3437, 10.3609, 10.2954, 10.193, 10.2504, 10.1625, 10.1339, 10.116, 10.1229, 10.0766, 10.0636, 9.97424, 9.94809, 10.0141, 9.92153, 9.87278, 9.88732, 9.83062, 9.85061, 9.78667, 9.734, 9.78553, 9.62389]

In [23]:
include("Algo/Partition.jl")
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")

Sig = v0[:,1:80]';

n_clusters = 200;
n_it_Kmeans = 100;
n_updates = 100;


community = vec(zeros(N,1));
community0 = vec(zeros(N,1));
Q_best = 0;
n_c_best = 0;
n_c = 0;

# Probability mass for sampling the initial centroids
p = vec(d/s);
n_updates_best = 0;

for k =1:n_it_Kmeans

	community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
	n_c = length(unique(community0));

    	if Q>Q_best
		community = community0;
		Q_best = Q;
		n_c_best = n_c;
		n_updates_best = n_updates;

		
    	end
end	
  
	community = Reorder(community);

	print("Number of updates: ")
	print(n_updates_best)
	print("\n")
	print("Number of communities: ")
	print(n_c_best)
	print("\n")

	print("Modularity: ")
	print(Q_best)
	print("\n")

Number of updates: 1
Number of communities: 70
Modularity: 0.7029548007385154


In [3]:
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")
include("Algo/Partition.jl")

n_clusters = 200;
n_it_Kmeans = 500;
n_updates = 100;
community = vec(zeros(N,1));
community0 = vec(zeros(N,1));

Q_best_best = 0;
n_c_best_best = 0;
n_updates_best_best = 0;
r_best = 2;
for r=2:80
    
    Sig = v0[:,1:r]';
    Q_best = 0;
    n_c_best = 0;
    n_c = 0;
    # Probability mass for sampling the initial centroids
    p = vec(d/s);
    n_updates_best = 0;

    for k =1:n_it_Kmeans

        community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
        n_c = length(unique(community0));
            if Q>Q_best
            community = community0;
            Q_best = Q;
            n_c_best = n_c;
            n_updates_best = n_updates;
            end
    end	

    community = Reorder(community);
    
    if Q_best>Q_best_best 
        r_best = r;
        Q_best_best = Q_best;
        n_c_best_best = n_c_best;
        n_updates_best_best = n_updates_best;
    end
    
end


	print("Number of updates: ")
	print(n_updates_best_best )
	print("\n")
	print("Number of communities: ")
	print(n_c_best_best )
	print("\n")

	print("Modularity: ")
	print(Q_best_best )
	print("\n")
	print("n: ")
	print(r_best )
	print("\n")


Number of updates: 0
Number of communities: 95
Modularity: 0.5497292756095178
n: 59
